# **Save this file as studentid1_studentid2_lab2.ipynb**, please check this suffix when you upload your lab, especially when you have multiple copy's in the same folder!
(Your student-id is the number shown on your student card.)

E.g. if you work with 3 people, the notebook should be named:
12301230_3434343_1238938934_lab2.ipynb.

**This will be parsed by a regexp, so please double check your filename.**

Before you turn this problem in, please make sure everything runs correctly. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Note, that **you are not allowed to use Google Colab**.

**Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your names and email adresses below.**



In [ ]:
NAME = "Dániel László"
NAME2 = "Henning Bartsch"
NAME3 = ""
EMAIL = "daniel.laszlo@student.uva.nl"
EMAIL2 = "bartsch.henning@gmail.com"
EMAIL3 = ""

# Lab 2: Classification

### Machine Learning 1, November 2018

Notes on implementation:

* You should write your code and answers in this IPython Notebook: http://ipython.org/notebook.html. If you have problems, please contact your teaching assistant.
* Please write your answers right below the questions.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Use the provided test cells to check if your answers are correct
* **Make sure your output and plots are correct before handing in your assignment with Kernel -> Restart & Run All**

* **If possible, all your implementations should be vectorized and rely on loops as little as possible. Therefore for some questions, we give you a maximum number of loops that are necessary for an efficient implementation. This number refers to the loops in this particular function and does not count the ones in functions that are called from the function. You should not go above this number for the maximum number of points.**

$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\ba}{\mathbf{a}}$
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bv}{\mathbf{v}}$
$\newcommand{\bq}{\mathbf{q}}$
$\newcommand{\bp}{\mathbf{p}}$
$\newcommand{\bh}{\mathbf{h}}$
$\newcommand{\bI}{\mathbf{I}}$
$\newcommand{\bX}{\mathbf{X}}$
$\newcommand{\bT}{\mathbf{T}}$
$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bV}{\mathbf{V}}$

In [ ]:
%pylab inline
plt.rcParams["figure.figsize"] = [9,5]

import time
start = time.time()

In [ ]:
# This cell makes sure that you have all the necessary libraries installed

import sys
import platform
from importlib.util import find_spec, module_from_spec

def check_newer_version(version_inst, version_nec):
    version_inst_split = version_inst.split('.')
    version_nec_split = version_nec.split('.')
    for i in range(min(len(version_inst_split), len(version_nec_split))):
        if int(version_nec_split[i]) > int(version_inst_split[i]):
            return False
        elif int(version_nec_split[i]) < int(version_inst_split[i]):
            return True
    return True
        
    
module_list = [('jupyter', '1.0.0'), 
               ('matplotlib', '2.0.2'), 
               ('numpy', '1.13.1'), 
               ('python', '3.6.2'), 
               ('sklearn', '0.19.0'), 
               ('scipy', '0.19.1'), 
               ('nb_conda', '2.2.1')]

packages_correct = True
packages_errors = []

for module_name, version in module_list:
    if module_name == 'scikit-learn':
        module_name = 'sklearn'
    if module_name == 'pyyaml':
        module_name = 'yaml'
    if 'python' in module_name:
        python_version = platform.python_version()
        if not check_newer_version(python_version, version):
            packages_correct = False
            error = f'Update {module_name} to version {version}. Current version is {python_version}.'
            packages_errors.append(error) 
            print(error)
    else:
        spec = find_spec(module_name)
        if spec is None:
            packages_correct = False
            error = f'Install {module_name} with version {version} or newer, it is required for this assignment!'
            packages_errors.append(error) 
            print(error)
        else:
            x =__import__(module_name)
            if hasattr(x, '__version__') and not check_newer_version(x.__version__, version):
                packages_correct = False
                error = f'Update {module_name} to version {version}. Current version is {x.__version__}.'
                packages_errors.append(error) 
                print(error)

try:
    from google.colab import drive
    packages_correct = False
    error = """Please, don't use google colab!
It will make it much more complicated for us to check your homework as it merges all the cells into one."""
    packages_errors.append(error) 
    print(error)
except:
    pass

packages_errors = '\n'.join(packages_errors)

# Part 1. Multiclass logistic regression

Scenario: you have a friend with one big problem: she's completely blind. You decided to help her: she has a special smartphone for blind people, and you are going to develop a mobile phone app that can do _machine vision_ using the mobile camera: converting a picture (from the camera) to the meaning of the image. You decide to start with an app that can read handwritten digits, i.e. convert an image of handwritten digits to text (e.g. it would enable her to read precious handwritten phone numbers).

A key building block for such an app would be a function `predict_digit(x)` that returns the digit class of an image patch $\bx$. Since hand-coding this function is highly non-trivial, you decide to solve this problem using machine learning, such that the internal parameters of this function are automatically learned using machine learning techniques.

The dataset you're going to use for this is the MNIST handwritten digits dataset (`http://yann.lecun.com/exdb/mnist/`). You can download the data with scikit learn, and load it as follows:

In [ ]:
from sklearn.datasets import fetch_mldata
import os
# Fetch the data
try:
    mnist = fetch_mldata('MNIST original', data_home='.')
except Exception:
    raise FileNotFoundError('Please download mnist-original.mat from Canvas and put it in %s/mldata' % os.getcwd())
data, target = mnist.data, mnist.target.astype('int')
# Shuffle
indices = np.arange(len(data))
np.random.seed(123)
np.random.shuffle(indices)
data, target = data[indices].astype('float32'), target[indices]

# Normalize the data between 0.0 and 1.0:
data /= 255. 

# Split
x_train, x_valid, x_test = data[:50000], data[50000:60000], data[60000: 70000]
t_train, t_valid, t_test = target[:50000], target[50000:60000], target[60000: 70000]

MNIST consists of small 28 by 28 pixel images of written digits (0-9). We split the dataset into a training, validation and testing arrays. The variables `x_train`, `x_valid` and `x_test` are $N \times M$ matrices, where $N$ is the number of datapoints in the respective set, and $M = 28^2 = 784$ is the dimensionality of the data. The second set of variables `t_train`, `t_valid` and `t_test` contain the corresponding $N$-dimensional vector of integers, containing the true class labels.

Here's a visualisation of the first 8 digits of the trainingset:

In [ ]:
def plot_digits(data, num_cols, targets=None, shape=(28,28)):
    num_digits = data.shape[0]
    num_rows = int(num_digits/num_cols)
    for i in range(num_digits):
        plt.subplot(num_rows, num_cols, i+1)
        plt.imshow(data[i].reshape(shape), interpolation='none', cmap='Greys')
        if targets is not None:
            plt.title(int(targets[i]))
        plt.colorbar()
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    
plot_digits(x_train[0:40000:5000], num_cols=4, targets=t_train[0:40000:5000])

# HOOK plot digits

In [ ]:
def plot_digits_with_cmap(data, num_cols, targets=None, shape=(28,28), cmap='Greys', title=None):
    min_intensity = data.min()
    max_intensity = data.max()
    
    num_digits = data.shape[0]
    num_rows = int(num_digits/num_cols)
    
    for i in range(num_digits):
        plt.subplot(num_rows, num_cols, i+1)
        plt.imshow(data[i].reshape(shape), interpolation='none', vmin=min_intensity, vmax=max_intensity, cmap=cmap)
        if targets is not None:
            plt.title(int(targets[i]))
        plt.colorbar()
        plt.axis('off')
    plt.tight_layout()
    
    fig = plt.gcf()
    
    if title is not None:
        fig.suptitle(title, )
        
    plt.show()

In _multiclass_ logistic regression, the conditional probability of class label $j$ given the image $\bx$ for some datapoint is given by:

$ \log p(t = j \;|\; \bx, \bb, \bW) = \log q_j - \log Z$

where $\log q_j = \bw_j^T \bx + b_j$ (the log of the unnormalized probability of the class $j$), and $Z = \sum_k q_k$ is the normalizing factor. $\bw_j$ is the $j$-th column of $\bW$ (a matrix of size $784 \times 10$) corresponding to the class label, $b_j$ is the $j$-th element of $\bb$.

Given an input image, the multiclass logistic regression model first computes the intermediate vector $\log \bq$ (of size $10 \times 1$), using $\log q_j = \bw_j^T \bx + b_j$, containing the unnormalized log-probabilities per class. 

The unnormalized probabilities are then normalized by $Z$ such that $\sum_j p_j = \sum_j \exp(\log p_j) = 1$. This is done by $\log p_j = \log q_j - \log Z$ where $Z = \sum_i \exp(\log q_i)$. This is known as the _softmax_ transformation, and is also used as a last layer of many classifcation neural network models, to ensure that the output of the network is a normalized distribution, regardless of the values of second-to-last layer ($\log \bq$)

**Warning**: when computing $\log Z$, you are likely to encounter numerical problems. Save yourself countless hours of debugging and learn the [log-sum-exp trick](https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/ "Title").

The network's output $\log \bp$ of size $10 \times 1$ then contains the conditional log-probabilities $\log p(t = j \;|\; \bx, \bb, \bW)$ for each digit class $j$. In summary, the computations are done in this order:

$\bx \rightarrow \log \bq \rightarrow Z \rightarrow \log \bp$

Given some dataset with $N$ independent, identically distributed datapoints, the log-likelihood is given by:

$ \mathcal{L}(\bb, \bW) = \sum_{n=1}^N \mathcal{L}^{(n)}$

where we use $\mathcal{L}^{(n)}$ to denote the partial log-likelihood evaluated over a single datapoint. It is important to see that the log-probability of the class label $t^{(n)}$ given the image, is given by the $t^{(n)}$-th element of the network's output $\log \bp$, denoted by $\log p_{t^{(n)}}$:

$\mathcal{L}^{(n)} = \log p(t = t^{(n)} \;|\; \bx = \bx^{(n)}, \bb, \bW) = \log p_{t^{(n)}} = \log q_{t^{(n)}} - \log Z^{(n)}$

where $\bx^{(n)}$ and $t^{(n)}$ are the input (image) and class label (integer) of the $n$-th datapoint, and $Z^{(n)}$ is the normalizing constant for the distribution over $t^{(n)}$.


## 1.1 Gradient-based stochastic optimization
### 1.1.1 Derive gradient equations (20 points)

Derive the equations for computing the (first) partial derivatives of the log-likelihood w.r.t. all the parameters, evaluated at a _single_ datapoint $n$.

You should start deriving the equations for $\frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}$ for each $j$. For clarity, we'll use the shorthand $\delta^q_j = \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}$.

For $j = t^{(n)}$:
$$
\delta^q_j
= \frac{\partial \log q_{t^{(n)}}}{\partial \log q_j}
-
\frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j} 
= 1
-
\frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j} 
$$

For $j \neq t^{(n)}$:
$$
\delta^q_j
= \frac{\partial \log q_{t^{(n)}}}{\partial \log q_j}
-
\frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j} 
=0 - \frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j}
$$

Complete the above derivations for $\delta^q_j$ by furtherly developing $\frac{\partial \log Z}{\partial Z}$ and $\frac{\partial Z}{\partial \log q_j}$. Both are quite simple. For these it doesn't matter whether $j = t^{(n)}$ or not.



For $j = t^{(n)}$:
\begin{align}
    \delta^q_j
    &= 1 -
    \frac{1}{Z} 
    \frac{\partial Z}{\partial \log q_j}
    = 1 -
    \frac{1}{Z} 
    \frac{\partial Z}{\partial q_j} 
    \frac{\partial q_j}{\partial \log q_j}
    = 1 -
    \frac{1}{Z} 
    \frac{\partial \sum_i q_i}{\partial q_j} 
    \frac{\partial q_j}{\partial \log q_j} 
    = 1 -
    \frac{1}{Z} 
    1
    \frac{\partial q_j}{\partial \log q_j} 
    = 1 -
    \frac{1}{Z}
    \frac{\partial \exp(\log q_j)}{\partial \log q_j} 
    = 1 -
    \frac{1}{Z}
    \exp(\log q_j) \\
    &= 1 -
    \frac{1}{Z}
    q_j
\end{align}

For $j \neq t^{(n)}$:
\begin{align}
    \delta^q_j
    &= 0 -
    \frac{1}{Z}
    q_j
\end{align}
YOUR ANSWER HERE

Given your equations for computing the gradients $\delta^q_j$ it should be quite straightforward to derive the equations for the gradients of the parameters of the model, $\frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}}$ and $\frac{\partial \mathcal{L}^{(n)}}{\partial b_j}$. The gradients for the biases $\bb$ are given by:

$
\frac{\partial \mathcal{L}^{(n)}}{\partial b_j}
= \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}
\frac{\partial \log q_j}{\partial b_j}
= \delta^q_j
\cdot 1
= \delta^q_j
$

The equation above gives the derivative of $\mathcal{L}^{(n)}$ w.r.t. a single element of $\bb$, so the vector $\nabla_\bb \mathcal{L}^{(n)}$ with all derivatives of $\mathcal{L}^{(n)}$ w.r.t. the bias parameters $\bb$ is: 

$
\nabla_\bb \mathcal{L}^{(n)} = \mathbf{\delta}^q
$

where $\mathbf{\delta}^q$ denotes the vector of size $10 \times 1$ with elements $\mathbf{\delta}_j^q$.

The (not fully developed) equation for computing the derivative of $\mathcal{L}^{(n)}$ w.r.t. a single element $W_{ij}$ of $\bW$ is:

$
\frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}} =
\frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}
\frac{\partial \log q_j}{\partial W_{ij}}
= \mathbf{\delta}_j^q
\frac{\partial \log q_j}{\partial W_{ij}}
$

What is $\frac{\partial \log q_j}{\partial W_{ij}}$? Complete the equation above.

If you want, you can give the resulting equation in vector format ($\nabla_{\bw_j} \mathcal{L}^{(n)} = ...$), like we did for $\nabla_\bb \mathcal{L}^{(n)}$.

YOUR ANSWER HERE

\begin{align}
    \frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}} =
    \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}
    \frac{\partial \log q_j}{\partial W_{ij}}
    = \mathbf{\delta}_j^q
    \frac{\partial \log q_j}{\partial W_{ij}}
    = \mathbf{\delta}_j^q
    \frac{\partial ( \mathbf{w}_j^T \mathbf{x}^{(n)} + b_j )}{\partial W_{ij}}
    = \mathbf{\delta}_j^q
    x_i^{(n)}
\end{align}

\begin{align}
\nabla_{\bw_j} \mathcal{L}^{(n)} = \mathbf{\delta}_j^q \mathbf{x}^{(n)} \\
\nabla_{\bW} \mathcal{L}^{(n)} = \mathbf{x}^{(n)} (\mathbf{\delta}^q)^T
\end{align}

### 1.1.2 Implement gradient computations (15 points)

Implement the gradient calculations you derived in the previous question. Write a function `logreg_gradient(x, t, w, b)` that returns the gradients $\nabla_{\bw_j} \mathcal{L}^{(n)}$ (for each $j$) and $\nabla_{\bb} \mathcal{L}^{(n)}$, i.e. the first partial derivatives of the log-likelihood w.r.t. the parameters $\bW$ and $\bb$, evaluated at a single datapoint (`x`, `t`).
The computation will contain roughly the following intermediate variables:

$
\log \bq \rightarrow Z \rightarrow \log \bp\,,\, \mathbf{\delta}^q
$

followed by computation of the gradient vectors $\nabla_{\bw_j} \mathcal{L}^{(n)}$ (contained in a $784 \times 10$ matrix) and $\nabla_{\bb} \mathcal{L}^{(n)}$ (a $10 \times 1$ vector).

For maximum points, ensure the function is numerically stable.


In [ ]:
DEBUG = False

In [ ]:
# IMPORTANT! Do not call this with a batch of xs
def get_log_q_and_log_Z(x, t, w, b):
    log_q = x @ w + b
    # log sum exp log_q = a + log sum exp (log_q - a)
    a = np.max(log_q)
    S = np.sum(np.exp(log_q - a))
    log_Z = a + np.log(S)
    log_p = log_q - log_Z
    return log_q, log_Z

In [ ]:
# 1.1.2 Compute gradient of log p(t|x;w,b) wrt w and b
def logreg_gradient(x, t, w, b):
    log_q, log_Z = get_log_q_and_log_Z(x, t, w, b)
    log_p = log_q - log_Z
    select_t = np.zeros_like(log_q)
    select_t[:, t] = 1
    delta_q = - 1 / np.exp(log_Z) * np.exp(log_q) + select_t
    
    dL_db = delta_q
    dL_dw = x.T @ delta_q
    if DEBUG:
        print("x.T.shape:", x.T.shape)
        print("delta_q.squeeze().shape:", delta_q.squeeze().shape)
        print("dL_db.shape:", dL_db.shape)
        print("dL_dw.shape:", dL_dw.shape)
    
    # here the statement contains logp[:,t] where logp is meant tas a matrix of shape 1x10
    
    return log_p[:,t].squeeze(), dL_dw, dL_db.squeeze()


In [ ]:
def logreg(x, t, w, b):
    log_q, log_Z = get_log_q_and_log_Z(x, t, w, b)
    return log_q - log_Z

In [ ]:
# Hidden tests for efficiency

In [ ]:
np.random.seed(123)
# scalar, 10 X 768  matrix, 10 X 1 vector
w = np.random.normal(size=(28*28,10), scale=0.001)
# w = np.zeros((784,10))
b = np.zeros((10,))

# test gradients, train on 1 sample
logpt, grad_w, grad_b = logreg_gradient(x_train[0:1,:], t_train[0:1], w, b)

print("Test gradient on one point")
print("Log Likelihood:\t", logpt)
print("\nGrad_W_ij\t",grad_w.shape,"matrix")
print("Grad_W_ij[0,152:158]=\t", grad_w[152:158,0])
print("\nGrad_B_i shape\t",grad_b.shape,"vector")
print("Grad_B_i=\t", grad_b.T)
print("i in {0,...,9}; j in M")

assert logpt.shape == (), logpt.shape
assert grad_w.shape == (784, 10), grad_w.shape
assert grad_b.shape == (10,), grad_b.shape




In [ ]:
# It's always good to check your gradient implementations with finite difference checking:
# Scipy provides the check_grad function, which requires flat input variables.
# So we write two helper functions that provide the gradient and output with 'flat' weights:
from scipy.optimize import check_grad

np.random.seed(123)
# scalar, 10 X 768  matrix, 10 X 1 vector
w = np.random.normal(size=(28*28,10), scale=0.001)
# w = np.zeros((784,10))
b = np.zeros((10,))

def func(w):
    logpt, grad_w, grad_b = logreg_gradient(x_train[0:1,:], t_train[0:1], w.reshape(784,10), b)
    return logpt
def grad(w):
    logpt, grad_w, grad_b = logreg_gradient(x_train[0:1,:], t_train[0:1], w.reshape(784,10), b)
    return grad_w.flatten()
finite_diff_error = check_grad(func, grad, w.flatten())
print('Finite difference error grad_w:', finite_diff_error)
assert finite_diff_error < 1e-3, 'Your gradient computation for w seems off'

def func(b):
    logpt, grad_w, grad_b = logreg_gradient(x_train[0:1,:], t_train[0:1], w, b)
    return logpt
def grad(b):
    logpt, grad_w, grad_b = logreg_gradient(x_train[0:1,:], t_train[0:1], w, b)
    return grad_b.flatten()
finite_diff_error = check_grad(func, grad, b)
print('Finite difference error grad_b:', finite_diff_error)
assert finite_diff_error < 1e-3, 'Your gradient computation for b seems off'



In [ ]:
# DO NOT REMOVE THIS CELL!
# It contains hidden tests



### 1.1.3 Stochastic gradient descent (15 points)

Write a function `sgd_iter(x_train, t_train, w, b)` that performs one iteration of stochastic gradient descent (SGD), and returns the new weights. It should go through the trainingset once in randomized order, call `logreg_gradient(x, t, w, b)` for each datapoint to get the gradients, and update the parameters **using a small learning rate of `1e-6`**. Note that in this case we're maximizing the likelihood function, so we should actually performing gradient ___ascent___... For more information about SGD, see Bishop 5.2.4 or an online source (i.e. https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

In [ ]:
def print_shapes(**kwargs):
    for i, j in kwargs.items():
        print(i, 'shape:', j.shape)

In [ ]:
print_shapes(**dict(x_train=x_train, t_train=t_train, w=w, b=b))

In [ ]:
log_likelihood_of_dataset(x_train, t_train, w, b, aggregate=False)

In [ ]:
def sgd_iter(x_train, t_train, W, b):
    lr = 1e-4
    if DEBUG:
        print_shapes(x_train, t_train, W, b)
    
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    
    log_p_train = 0
    
    for i in indices:
        x = x_train[i:i+1, :]
        t = t_train[i]
        # TODO Ask Thomas if we should sum these log_ps, or recalculate the whole likelihood after a full epoch
        log_p, dL_dW, dL_db = logreg_gradient(x, t, W, b)
        log_p_train += log_p
    
        # TODO we are modifying the original here, should this be the case?
        # Ascent, so using +
        W += lr * dL_dW
        b += lr * dL_db

    return log_p_train, W, b

In [ ]:
# Hidden tests for efficiency

In [ ]:
# Sanity check:
np.random.seed(1243)
w = np.zeros((28*28, 10))
b = np.zeros(10)
    
logp_train, W, b = sgd_iter(x_train[:5], t_train[:5], w, b)




## 1.2. Train

### 1.2.1 Train (12 points)
Perform SGD on the training set. Plot (in one graph) the conditional log-probability of the training set and validation set after each iteration. (6 points)

Instead of running SGD for a fixed number of steps, run it until convergence. Think of a reasonable criterion for determining convergence. As a reference: choose a criterion such that the algorithm terminates in less than 15 iterations over the training set. (2 points)

Make sure your implementation (in particular, the output of the conditional log-probability of the training set and validation set) is independent of the size of the dataset. (2 points)

In [ ]:
def stopping_criteria(valid_results_avg, eps=0.1):
    return abs(valid_results_avg[-2]-valid_results_avg[-1]) < eps

In [ ]:
def log_likelihood_of_dataset(X, t, W, b, aggregate=True):
    log_q = X @ W + b
    
    # log sum exp log_q = a + log sum exp (log_q - a)
    a = np.max(log_q, axis=1)
    S = np.sum(np.exp(log_q - a[:, None]), axis=1)
    log_Z = a + np.log(S)
    log_p = log_q - log_Z[:, np.newaxis]
    
    if DEBUG:
        print_shapes(**dict(log_q=log_q, a_np_newaxis=a[:, np.newaxis], S=S, log_Z=log_Z, log_p=log_p))
    
    #https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing
    x_idx = np.arange(log_p.shape[0])
    
    log_p_t = log_p[x_idx, t]
    if aggregate:
        return np.sum(log_p_t)
    else:
        return log_p_t

In [ ]:
class Stats:
    def __init__(self, train_size, valid_size, DEBUG=False):
        self.DEBUG = DEBUG
        self.train_size = train_size
        self.valid_size = valid_size
        
        self.train_results = []
        self.valid_results = []

    def append(self, logp_train, logp_valid):
        if self.DEBUG:
            print(logp_train)
            print(logp_valid)
        self.train_results.append(logp_train)
        self.valid_results.append(logp_valid)
        
    def stopping_criteria(self, eps):
        try:
            avg2 = self.valid_results[-2] / self.valid_size
            avg1 = self.valid_results[-1] / self.valid_size
            if self.DEBUG:
                print('Criteria input:', abs(avg2 - avg1))
            return abs(avg2 - avg1) < eps
        except IndexError:
            return False
        
    def get_avgs(self):
        return np.array(self.train_results) / self.train_size, np.array(self.valid_results) / self.valid_size

In [ ]:
stats = Stats(5, 5)
stats.get_avgs()

In [ ]:
# This is really ugly, but I don't want to plot inside the function...
# Also it only makes sense, if the test_sgd is really going to be called with
stats = Stats(len(x_train), len(x_valid))
def test_sgd(x_train, t_train, x_valid, t_valid, w, b):
    # YOUR CODE HERE

    count = 0
    
    # This is actually a do-while loop
    while True:
        print('Iteration count:', count); count+=1
        logp_train, w, b = sgd_iter(x_train, t_train, w, b)
        logp_valid = log_likelihood_of_dataset(x_valid, t_valid, w, b)
        
        stats.append(logp_train, logp_valid)
            
        if stats.stopping_criteria(eps=0.008):
            break
            
    return w, b

np.random.seed(1243)
w = np.zeros((28*28, 10))
b = np.zeros(10)
w, b = test_sgd(x_train, t_train, x_valid, t_valid, w, b)

In [ ]:
#%config InlineBackend.figure_format = 'png'
#%config InlineBackend.figure_format = 'svg'

In [ ]:
avg_train_log_p, avg_valid_log_p = stats.get_avgs()

plt.figure(figsize=(8, 5))
plt.title('Average conditional log-probabilities')
plt.xlabel('iteration')
plt.ylabel('conditional log-probability')
plt.plot(avg_train_log_p, label='Train set')
plt.plot(avg_valid_log_p, label='Validation set')
plt.legend()
plt.show()

In [ ]:
# Hidden tests for efficiency

### 1.2.2 Visualize weights (10 points)
Visualize the resulting parameters $\bW$ after a few iterations through the training set, by treating each column of $\bW$ as an image. If you want, you can use or edit the `plot_digits(...)` above.


In [ ]:
# YOUR CODE HERE
plt.figure(figsize=(16,5))
plot_digits_with_cmap(w.T, 5, targets=np.arange(10), cmap='afmhot')
#plot_digits_with_cmap(w.T, 5, cmap='viridis')
#plot_digits_with_cmap(w.T, 5, cmap='ocean')
#plot_digits_with_cmap(w.T, 5, np.arange(10), cmap='jet')

**Describe in less than 100 words why these weights minimize the loss**
YOUR ANSWER HERE
The visualizations of the weights clearly resemble the actual numbers, that the weights belong to. This means, that the log probability of a class given an input image is going to be the highest, when the input image and the weight have high values on the same pixel-positions.

### 1.2.3. Visualize the 8 hardest and 8 easiest digits (10 points)
Visualize the 8 digits in the validation set with the highest probability of the true class label under the model.
Also plot the 8 digits that were assigned the lowest probability.


In [ ]:
def get_n_most_likely(x_valid, t_valid, w, b, n=-8):
    '''
    If n is negative, this will get the n most likely,
    if n is positive, it will get the n least likely images (hardest to recognize)
    '''
    logp_valid = log_likelihood_of_dataset(x_valid, t_valid, w, b, aggregate=False)
    
    if n < 0:
        highest_indices = np.argpartition(logp_valid, n)[n:]
    else:
        highest_indices = np.argpartition(logp_valid, n)[:n]

    highest_logp = logp_valid[highest_indices]

    easiest_imgs, easiest_targets = x_valid[highest_indices], t_valid[highest_indices]

    sorted_indices = np.argsort(highest_logp)
    sorted_imgs, sorted_targets = easiest_imgs[sorted_indices], easiest_targets[sorted_indices]

    return sorted_imgs, sorted_targets

In [ ]:
# YOUR CODE HERE
easy_sorted_imgs, easy_sorted_targets = get_n_most_likely(x_valid, t_valid, w, b, n=-8)

plt.figure(figsize(12,5))

plot_digits_with_cmap(easy_sorted_imgs, 4, targets=easy_sorted_targets, cmap='afmhot', title='Easiest to recognize images')
plt.show()

hard_sorted_imgs, hard_sorted_targets = get_n_most_likely(x_valid, t_valid, w, b, n=8)

plt.figure(figsize(12,5))
plot_digits_with_cmap(hard_sorted_imgs, 4, targets=hard_sorted_targets, cmap='afmhot', title='Hardest to recognize images')
plt.show()



In [ ]:
true_weights = w[:, easy_sorted_targets]

result = easy_sorted_imgs.T * true_weights

plt.figure(figsize(12,5))
plot_digits_with_cmap(result.T, 4, targets=easy_sorted_targets, cmap='afmhot', title='Elementwise multiplication of true weights and image')
plt.show()


true_weights = w[:, hard_sorted_targets]

result = hard_sorted_imgs.T * true_weights

plt.figure(figsize(12,5))
plot_digits_with_cmap(result.T, 4, targets=hard_sorted_targets, cmap='afmhot')
plt.show()


Ask yourself if these results make sense. Explain in no more then two sentences what it means that a digit is hard to classify.

YOUR ANSWER HERE

# Part 2. Multilayer perceptron


You discover that the predictions by the logistic regression classifier are not good enough for your application: the model is too simple. You want to increase the accuracy of your predictions by using a better model. For this purpose, you're going to use a multilayer perceptron (MLP), a simple kind of neural network. The perceptron will have a single hidden layer $\bh$ with $L$ elements. The parameters of the model are $\bV$ (connections between input $\bx$ and hidden layer $\bh$), $\ba$ (the biases/intercepts of $\bh$), $\bW$ (connections between $\bh$ and $\log q$) and $\bb$ (the biases/intercepts of $\log q$).

The conditional probability of the class label $j$ is given by:

$\log p(t = j \;|\; \bx, \bb, \bW) = \log q_j - \log Z$

where $q_j$ are again the unnormalized probabilities per class, and $Z = \sum_j q_j$ is again the probability normalizing factor. Each $q_j$ is computed using:

$\log q_j = \bw_j^T \bh + b_j$

where $\bh$ is a $L \times 1$ vector with the hidden layer activations (of a hidden layer with size $L$), and $\bw_j$ is the $j$-th column of $\bW$ (a $L \times 10$ matrix). Each element of the hidden layer is computed from the input vector $\bx$ using:

$h_j = \sigma(\bv_j^T \bx + a_j)$

where $\bv_j$ is the $j$-th column of $\bV$ (a $784 \times L$ matrix), $a_j$ is the $j$-th element of $\ba$, and $\sigma(.)$ is the so-called sigmoid activation function, defined by:

$\sigma(x) = \frac{1}{1 + \exp(-x)}$

Note that this model is almost equal to the multiclass logistic regression model, but with an extra 'hidden layer' $\bh$. The activations of this hidden layer can be viewed as features computed from the input, where the feature transformation ($\bV$ and $\ba$) is learned.

## 2.1 Derive gradient equations (20 points)

State (shortly) why $\nabla_{\bb} \mathcal{L}^{(n)}$ is equal to the earlier (multiclass logistic regression) case, and why $\nabla_{\bw_j} \mathcal{L}^{(n)}$ is almost equal to the earlier case.

Like in multiclass logistic regression, you should use intermediate variables $\mathbf{\delta}_j^q$. In addition, you should use intermediate variables $\mathbf{\delta}_j^h = \frac{\partial \mathcal{L}^{(n)}}{\partial h_j}$.

Given an input image, roughly the following intermediate variables should be computed:

$
\log \bq \rightarrow Z \rightarrow \log \bp \rightarrow \mathbf{\delta}^q \rightarrow \mathbf{\delta}^h
$

where $\mathbf{\delta}_j^h = \frac{\partial \mathcal{L}^{(n)}}{\partial \bh_j}$.

Give the equations for computing $\mathbf{\delta}^h$, and for computing the derivatives of $\mathcal{L}^{(n)}$ w.r.t. $\bW$, $\bb$, $\bV$ and $\ba$. 

You can use the convenient fact that $\frac{\partial}{\partial x} \sigma(x) = \sigma(x) (1 - \sigma(x))$.

We can consider the output layer as the same multiclass logistic regression seen before. The difference from this layer's point of view is only the input: $\bh$ (the feature-vector, that is learned by the hidden layer). 

$\nabla_{\bb} \mathcal{L}^{(n)}$: As this gradient didn't (directly) depend on the input (only through $\mathbf{\delta}^q$), we can use the same formula here.

$\nabla_{\bw_j} \mathcal{L}^{(n)}$: Similarly, this gradient depended only on $\mathbf{\delta}^q$ and the input, so we can get our new formula by replacing the previous input with the current one.


Let $\bm = \bV^T \bx + \ba$, and $m_j = \bv_j^T \bx + a_j$, where $\bv_j$ is the $\text{j}^{\text{th}}$ column of $\bV$.


\begin{align}
\frac{\partial \log q_k}{\partial h_j} &= \frac{\partial \bw_k^T \bh}{\partial h_j} = W_{jk} \\
\delta^h_j = \frac{\partial \mathcal{L}^{(n)}}{\partial h_j} = \sum_k \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_k} \frac{\partial \log q_k}{\partial h_j} &= \sum_k \delta^q_k W_{jk} \\
\end{align}

\begin{align}
\delta^m_j = \frac{\partial \mathcal{L}^{(n)}}{\partial m_j} = \sum_k \frac{\partial \mathcal{L}^{(n)}}{\partial h_k} \frac{\partial h_k}{\partial m_j} = \sum_k \delta^h_k \frac{\partial \sigma(m_k)}{\partial m_j} = \delta^h_j \frac{\partial \sigma(m_j)}{\partial m_j} = \delta^h_j \sigma(m_j)(1-\sigma(m_j))\\
\nabla_{V_{ij}} \mathcal{L}^{(n)} = \sum_k \delta^m_k \frac{\partial m_k}{\partial V_{ij}} = \sum_k \delta^m_k \frac{\partial \bv_k^T \bx}{\partial V_{ij}} = \delta^m_j \frac{\partial \bv_j^T \bx}{\partial V_{ij}} = \delta^m_j x_i
\end{align}


\begin{align}
\nabla_{\bb} \mathcal{L}^{(n)} &= \mathbf{\delta}^q \\
\nabla_{\bw_j} \mathcal{L}^{(n)} &= \mathbf{\delta}^q_j \bh^{(n)} \\
\nabla_{\bW} \mathcal{L}^{(n)} &= \bh^{(n)} (\mathbf{\delta}^q)^T \\
\nabla_{V_{ij}} \mathcal{L}^{(n)} &= \delta^m_j x_i \\
\nabla_{\bv_j} \mathcal{L}^{(n)} &= \mathbf{\delta}^m_j \bx^{(n)} \\
\nabla_{V} \mathcal{L}^{(n)} &= \bx (\delta^m)^T \\
\nabla_{a_i} \mathcal{L}^{(n)} &= \delta^m_j \\
\nabla_{\ba} \mathcal{L}^{(n)} &= \mathbf{\delta}^m
\end{align}

## 2.2 MAP optimization (10 points)

You derived equations for finding the _maximum likelihood_ solution of the parameters. Explain, in a few sentences, how you could extend this approach so that it optimizes towards a _maximum a posteriori_ (MAP) solution of the parameters, with a Gaussian prior on the parameters. 

YOUR ANSWER HERE

TODO

## 2.3. Implement and train a MLP (15 points)

Implement an MLP model with a single hidden layer of **20 neurons**. 
Train the model for **10 epochs**.
Test your implementation for learning rates of 1e-2, 1e-3 and 1e-4 and plot (in one graph) the conditional log-probability of the trainingset and validation set. 

For the best model plot the weights of the first layer for in epoch 0,4 and 9. 


- 10 points: Working MLP that learns with plots
- +5 points: Fast, numerically stable, vectorized implementation

In [ ]:
class Sigmoid:
    def __call__(self, x):
        self.x = x
        return 1.0 / (1.0 + np.exp(-x))
    
    def backward(self, delta):
        return delta * self(self.x)*(1-self(self.x))


In [ ]:
class Softmax:    
    def __call__(self, log_q):        
        a = np.max(log_q, axis=1, keepdims=True)
        S = np.sum(np.exp(log_q - a), axis=1, keepdims=True)
        log_Z = a + np.log(S)
        
        if DEBUG:
            print_shapes(**dict(log_q=log_q, a=a, S=S, logz=logz))
            
        log_p = log_q - log_Z
        self.log_q, self.log_p, self.log_Z = log_q, log_p, log_Z
        
        return log_p
    
    def backward(self, t):
        log_p, log_q, log_Z = self.log_p, self.log_q, self.log_Z
        select_t = np.zeros_like(log_q)
        select_t[:,t] = 1
        delta_q = - 1 / np.exp(log_Z) * np.exp(log_q) + select_t

        return -delta_q


In [ ]:
class Linear:
    def __init__(self, input_dim, out_dim):
        # Bias is incorporated
        self.w = np.random.normal(size=(input_dim + 1, out_dim))
        self.grad_w = np.zeros(self.w.shape)
    
    def prepend_bias_feature(self, x):
        shape = list(x.shape)
        shape[1] += 1
        y = np.zeros(shape)
        y[:, 0] = 1
        y[:, 1:] = x
        return y
    
    def __call__(self, x):
        x = self.prepend_bias_feature(x)
        m = x @ self.w
        
        self.x = x
        return m
    
    def backward(self, delta_q):        
        self.grad_w[:] = self.x.T @ delta_q
        
        grad_x = delta_q @ self.w.T
        if DEBUG:
            print_shapes(**dict(self_w_T=self.w.T, delta_q=delta_q, grad_x=grad_x))

        # Without the bias term
        return grad_x[:, 1:]
    
    def step(self, lr):
        self.w -= lr * self.grad_w


In [ ]:
# Write all helper functions here
class MLP:
    def __init__(self, input_size=28*28, hidden_layer_size=20, output_size=10):
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        
        # bias is incorporated into the weight matrix
        self.V = Linear(input_size, hidden_layer_size)
        self.activation1 = Sigmoid()
        self.W = Linear(hidden_layer_size, output_size)
        self.activation2 = Softmax()
        
    def forward(self, x):
        m = self.V(x)
        h = self.activation1(m)
        log_q = self.W(h)
        log_p = self.activation2(log_q)

        return log_p
        
    def backward(self, t):
        delta = self.activation2.backward(t)
        delta = self.W.backward(delta)
        delta = self.activation1.backward(delta)
        delta = self.V.backward(delta)
        
        return delta
    
    def step(self, lr):
        self.W.step(lr)
        self.V.step(lr)


In [ ]:
class SGD:
    def __init__(self, mlp, lr):
        self.mlp = mlp
        self.lr = lr
        
    def sgd_iter(self, x_train, t_train):
        assert len(x_train) == len(t_train), 'The number of inputs and targets are not the same'
        
        indices = np.arange(len(x_train))
        np.random.shuffle(indices)
        
        log_p_train = 0
        
        for i in indices:
            x = x_train[i:i+1, :]
            t = t_train[i]

            log_p = self.mlp.forward(x)
            log_p_train += log_p.squeeze()[t]
            
            self.mlp.backward(t)
            self.mlp.step(self.lr)
            
        return log_p_train
    
    def sgd_eval(self, x_valid, t_valid):
        assert len(x_valid) == len(t_valid), 'The number of inputs and targets are not the same'
        
        logp = self.mlp.forward(x_valid)
        x_idx = np.arange(len(logp))
        
        log_likelihoods = logp[x_idx, t_valid]
        return np.sum(log_likelihoods)
        

In [ ]:
from contextlib import contextmanager

@contextmanager
def manually_stoppable_block():
    try:
        yield
    except KeyboardInterrupt:
        print('Block was stopped manually')


In [ ]:
def test_mlp(sgd, x_train, t_train, x_valid, t_valid, epochs=10, checkpointer=lambda w, idx: None, DEBUG=False):
    stats = Stats(len(x_train), len(x_valid), DEBUG=DEBUG)

    stats.append(sgd.sgd_eval(x_train, t_train), sgd.sgd_eval(x_valid, t_valid))
    
    with manually_stoppable_block():
        for e in range(epochs):
            print('Iteration count:', e)
            checkpointer(sgd.mlp.V.w, e)
            logp_train = sgd.sgd_iter(x_train, t_train)
            logp_valid = sgd.sgd_eval(x_valid, t_valid)
            stats.append(logp_train, logp_valid)
            
    return stats

In [ ]:
class WeightSaver:
    def __init__(self, at_timesteps):
        self.at_timesteps = at_timesteps
        self.chkpts = []
        
    def __call__(self, w, idx):
        if idx in self.at_timesteps:
            print('Saving weights at timestep', idx)
            self.chkpts.append(w.copy())

In [ ]:
# Hidden tests for efficiency

In [ ]:
from collections import namedtuple
TrainConfig = namedtuple("TrainConfig", ["lr", "mlp", "sgd", "weights", "stats"])

In [ ]:
# Write training code here:
# Training code
train_configs = []
lrs = [1e-2, 1e-3, 1e-4]
for lr in lrs:
    print('lr:', lr)
    mlp = MLP()
    sgd = SGD(mlp, lr=lr)
    weights = WeightSaver([0, 4, 9])
    stats = test_mlp(sgd, x_train, t_train, x_valid, t_valid, checkpointer=weights)
    train_configs.append(TrainConfig(lr=lr, mlp=mlp, sgd=sgd, weights=weights, stats=stats))

# Plot the conditional loglikelihoods for the train and validation dataset after every iteration.
# See below


import pickle

In [ ]:
with open('train_configs.p', 'wb') as f:
    pickle.dump(train_configs, f, )

with open('train_configs.p', 'rb') as f:
    train_configs = pickle.load(f)

In [ ]:
# plot the train and validation logp for all three learning rates in one figure
# YOUR CODE HERE
for config in train_configs:
    train_avgs, valid_avgs = config.stats.get_avgs()
    #train_avgs, valid_avgs = config.stats.train_results, config.stats.valid_results
    plt.plot(train_avgs, label='lr={}, train '.format(config.lr))
    plt.plot(valid_avgs, label='lr={}, validation'.format(config.lr))
    
plt.xlabel('epoch')
plt.ylabel('avg log likelihood')
plt.legend()
plt.title('Average conditional log-likelihoods during training')

plt.show()
    

In [ ]:
# Plot the weights of the first layer.
# YOUR CODE HERE
plt.figure(figsize=(10, 10))
plot_digits_with_cmap(train_configs[0].mlp.V.w[1:].T, 4, cmap='afmhot')


### 2.3.1. Explain the learning curves (5 points)
In less than 80 words, explain the observed behaviour for the different learning rates.

YOUR ANSWER HERE

TODO

### 2.3.2. Explain the weights (5 points)
In less than 80 words, explain how and why the weights of the hidden layer of the MLP differ from the logistic regression model, and relate this to the stronger performance of the MLP.

In [ ]:
w = train_configs[0].weights

In [ ]:
w.chkpts[0].shape

In [ ]:
47100 / 784

In [ ]:
W = np.stack(w.chkpts, axis=-1)

In [ ]:
W[1:, :].shape

In [ ]:
W_p = W[1:, :].reshape(784, -1)

In [ ]:
# Plot the weights of the first layer for the best model 

# YOUR CODE HERE
plt.figure(figsize=(8, 40))
plot_digits_with_cmap(W_p.T, 3, cmap='afmhot')


YOUR ANSWER HERE

TODO

In [ ]:
# Plot the weights of the first layer for the best model 

# YOUR CODE HERE
plt.figure(figsize=(10, 10))
plot_digits_with_cmap(train_configs[0].mlp.V.w[1:].T, 4, cmap='afmhot')


### 2.3.2. Different activation functions (10 points)
In the task above we use a sigmoid as an activation function.
Two other popular choices for activation functions are tanh and the rectified linear unit (ReLU). The ReLU is defined as:

$$f(x) = \max(0.,x)$$

You already derived the derivative of the softmax function above. Here, write down the derivative for both the tanh and the ReLU function. Furthermore, for all three, plot the function and its derivative in a range $x\in[-3,3]$

Write down the derivative of ReLU and tanh w.r.t. their respective argument:

YOUR ANSWER HERE
\begin{align}
    \frac{\partial \mbox{ReLU}(x)}{\partial x} = \frac{\partial \mbox{max}(0, x)}{\partial x} = 
    \begin{cases}
        0 & \mbox{if } x < 0 \\ 
        1 & \mbox{if } x > 0 
    \end{cases}
\end{align}

\begin{align}
    \frac{\partial tanh(x)}{\partial x} = 
    \frac{\partial}{\partial x} \left(\frac{e^x - e^{-x}}{e^x + e^{-x}}\right) = 
    \frac{(e^x + e^{-x}) (e^x + e^{-x}) - (e^x - e^{-x}) (e^x - e^{-x}) }{(e^x + e^{-x})^2} = 
    1 - \tanh^2(x)
\end{align}

Name two properties that you would like your activation function to have (one sentence each). Why are they important?

YOUR ANSWER HERE

TODO

In [ ]:
class Tanh:
    def __call__(self, x):
        self.x = x
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    
    def backward(self, delta):
        return (1 - self(self.x) ** 2) * delta

In [ ]:
class ReLU:
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)
    
    def backward(self, delta):
        return (self.x > 0).astype(float) * delta

In [ ]:
tanh = Tanh()
sigmoid = Sigmoid()
relu = ReLU()

In [ ]:
# plot the function and the derivative for the activations sigmoid, tanh and ReLU.

# YOUR CODE HERE
test_data = np.arange(-10, 10, 0.001)
plt.figure(figsize=(12, 8))
plt.plot(test_data, sigmoid(test_data), label='σ(x)')
plt.plot(test_data, sigmoid.backward(1), label='∇σ(x)')
plt.plot(test_data, tanh(test_data), label='tanh(x)')
plt.plot(test_data, tanh.backward(1), label='∇tanh(x)')
plt.plot(test_data, relu(test_data), label='ReLU(x)')
plt.plot(test_data, relu.backward(1), label='∇ReLU(x)')
plt.legend()
plt.xlim(xmin=-3, xmax=3)
plt.ylim(ymin=-2, ymax=2)
plt.title("Activation functions")

Now that you plotted the activations and derivatives, which activation do you think is the best? Why would you choose this activation function? For your answer consider what you named as essential properties for an activation function above. Keep your answer short at no more then 3 sentences.

YOUR ANSWER HERE

TODO

In [ ]:
print('Notebook ran in {:2.3} minutes.'.format((time.time()-start)/60))